In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, LeakyReLU, Dropout
from sklearn.model_selection import KFold
import pandas as pd
from IPython.display import FileLink

# ✅ Configuration
MODEL_NAME = "CustomVGG16"
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 75  # You can set to 100 for longer training
NUM_FOLDS = 5
SEED = 42
IMG_DIR = "/kaggle/input/augmented-dr-dataset/Augmented_DR_Dataset"
MODEL_SAVE_DIR = f"/kaggle/working/{MODEL_NAME}"
RESULTS_CSV_PATH = f"{MODEL_SAVE_DIR}/training_results.csv"
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# ✅ Load Dataset
image_files = []
for root, _, files in os.walk(IMG_DIR):
    for file in files:
        if file.endswith('.png'):
            image_files.append(os.path.join(root, file))

print(f"Found {len(image_files)} image files.")

# ✅ Extract labels from subfolder names
labels = [os.path.basename(os.path.dirname(f)) for f in image_files]

# ✅ Create DataFrame
data_df = pd.DataFrame({"filename": image_files, "label": labels})
print(f"Total Images Found: {len(data_df)}")
class_names = sorted(data_df["label"].unique())

# ✅ Define Custom VGG16-based Model
def build_custom_vgg16(input_shape=(128, 128, 3), num_classes=5):
    base_model = VGG16(include_top=False, weights=None, input_shape=input_shape)

    model = Sequential([
        base_model,
        Flatten(),
        Dense(128),
        BatchNormalization(),
        LeakyReLU(alpha=0.01),
        Dropout(0.5),

        Dense(64),
        BatchNormalization(),
        LeakyReLU(alpha=0.01),
        Dropout(0.5),

        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# ✅ K-Fold Cross Validation
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED)
results = []

# ✅ Training Loop
for fold, (train_idx, val_idx) in enumerate(kf.split(data_df)):
    print(f"\n========= 🏆 Fold {fold+1}/{NUM_FOLDS} =========")

    train_df, val_df = data_df.iloc[train_idx], data_df.iloc[val_idx]

    # ✅ Data Augmentation
    train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, horizontal_flip=True)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_datagen.flow_from_dataframe(train_df, x_col="filename", y_col="label", 
                                                  target_size=IMG_SIZE, batch_size=BATCH_SIZE, 
                                                  class_mode="categorical")
    val_gen = val_datagen.flow_from_dataframe(val_df, x_col="filename", y_col="label", 
                                              target_size=IMG_SIZE, batch_size=BATCH_SIZE, 
                                              class_mode="categorical")

    # ✅ Build Model
    model = build_custom_vgg16(input_shape=(*IMG_SIZE, 3), num_classes=len(class_names))

    # ✅ Checkpointing
    fold_model_name = f"{MODEL_NAME}_fold_{fold+1}.keras"
    checkpoint_path = os.path.join(MODEL_SAVE_DIR, fold_model_name)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_accuracy', mode='max')

    # ✅ Train Model
    history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[checkpoint])

    # ✅ Best Results
    best_val_acc = max(history.history['val_accuracy'])
    best_val_loss = min(history.history['val_loss'])

    results.append({
        "Model": MODEL_NAME,
        "Fold": fold + 1,
        "Best Val Accuracy": best_val_acc,
        "Best Val Loss": best_val_loss
    })

# ✅ Save Results CSV
results_df = pd.DataFrame(results)
results_df.to_csv(RESULTS_CSV_PATH, index=False)
print(f"\n📊 Training results saved to: {RESULTS_CSV_PATH}")
print(f"🎯 Best models saved at: {MODEL_SAVE_DIR}")

# ✅ Zip and Provide Download Link
shutil.make_archive(MODEL_SAVE_DIR, 'zip', MODEL_SAVE_DIR)
display(FileLink(f"{MODEL_SAVE_DIR}.zip"))


Found 3493 image files.
Total Images Found: 3493

========= 🏆 Fold 1/5 =========
Found 2794 validated image filenames belonging to 5 classes.
Found 699 validated image filenames belonging to 5 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/75


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


88/88 ━━━━━━━━━━━━━━━━━━━━ 76s 585ms/step - accuracy: 0.2760 - loss: 1.9636 - val_accuracy: 0.4664 - val_loss: 1.2686
Epoch 2/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 21s 218ms/step - accuracy: 0.5942 - loss: 1.0658 - val_accuracy: 0.6853 - val_loss: 0.8579
Epoch 3/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 19s 204ms/step - accuracy: 0.6721 - loss: 0.9715 - val_accuracy: 0.6481 - val_loss: 1.0123
Epoch 4/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 213ms/step - accuracy: 0.6869 - loss: 0.8726 - val_accuracy: 0.6767 - val_loss: 1.6276
Epoch 5/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 207ms/step - accuracy: 0.7234 - loss: 0.8175 - val_accuracy: 0.6767 - val_loss: 0.8530
Epoch 6/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 211ms/step - accuracy: 0.7273 - loss: 0.7887 - val_accuracy: 0.5494 - val_loss: 1.0234
Epoch 7/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 21s 222ms/step - accuracy: 0.7376 - loss: 0.7766 - val_accuracy: 0.7110 - val_loss: 0.8202
Epoch 8/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 21s 220ms/step - accuracy: 0.7589 - loss: 0.7425 - val_accuracy: 0.728

...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/75


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


88/88 ━━━━━━━━━━━━━━━━━━━━ 30s 248ms/step - accuracy: 0.3533 - loss: 1.7491 - val_accuracy: 0.5258 - val_loss: 1.4044
Epoch 2/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 21s 228ms/step - accuracy: 0.6247 - loss: 1.0464 - val_accuracy: 0.7235 - val_loss: 0.8726
Epoch 3/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 210ms/step - accuracy: 0.6721 - loss: 0.9087 - val_accuracy: 0.4900 - val_loss: 1.7977
Epoch 4/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 19s 203ms/step - accuracy: 0.6822 - loss: 0.8695 - val_accuracy: 0.5244 - val_loss: 1.4636
Epoch 5/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 209ms/step - accuracy: 0.6910 - loss: 0.8641 - val_accuracy: 0.6490 - val_loss: 0.9406
Epoch 6/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 214ms/step - accuracy: 0.7083 - loss: 0.8432 - val_accuracy: 0.7292 - val_loss: 0.7972
Epoch 7/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 19s 202ms/step - accuracy: 0.7180 - loss: 0.7925 - val_accuracy: 0.7049 - val_loss: 1.1882
Epoch 8/75
88/88 ━━━━━━━━━━━━━━━━━━━━ 20s 207ms/step - accuracy: 0.7224 - loss: 0.7911 - val_accuracy: 0.646

/kaggle/working/CustomVGG16.zip